In [1]:
from __future__ import division

In [2]:
import tensorflow as tf
import tflearn

In [3]:
import numpy as np

In [4]:
from tensorflow.python import debug as tf_debug

In [5]:
sess = tf.Session()
# sess = tf_debug.LocalCLIDebugWrapperSession(sess)
lr = tf.placeholder(tf.float32, shape=[])
optim = tf.train.AdamOptimizer(learning_rate=lr)
# sgd_optim = tf.train.GradientDescentOptimizer(learning_rate=lr)

In [6]:
# sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)

In [7]:
from tqdm import tqdm

In [8]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [9]:
# import BayesianNN

In [10]:
# bnn = BayesianNN.BayesianFC([4,32,32,1])

In [11]:
import Bayesian_FC_Layer, Bayesian_Net

In [12]:
def bbnet(i, o, mean_init, std_init, prior_mean, prior_std, activation=tf.nn.relu):
    return Bayesian_FC_Layer.Bayesian_FC(i, o, 
                                         qw_mean_initial=mean_init, 
                                         qw_std_initial=std_init, 
                                         qb_std_initial=std_init,
                                         pw_mean=prior_mean,
                                         pw_sigma=prior_std,
                                         pb_mean=prior_mean,
                                         pb_sigma=prior_std,
                                         activation=activation
                                         )

In [13]:
std_init = 0.1
mean_init = 0.1
prior_mean = 0.0
prior_std = 1.0
l1 = bbnet(4, 32, mean_init, std_init, prior_mean, prior_std)
l2 = bbnet(32, 32, mean_init, std_init, prior_mean, prior_std)
l3 = bbnet(32, 1, mean_init, std_init, prior_mean, prior_std, activation=tf.identity)
# l1 = Bayesian_FC_Layer.Bayesian_FC(4, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l2 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l3 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l4 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l5 = Bayesian_FC_Layer.Bayesian_FC(32, 1, activation=tf.identity, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)

In [14]:
bnet = Bayesian_Net.Bayesian_Net([l1, l2, l3])

In [15]:
def test_func(x):
    e = np.random.normal(0,0.01)
    return x + 0.3*np.sin(2 *np.pi * (x + e)) + 0.3*np.sin(4 * np.pi*(x + e)) + e

In [16]:
data = [(x, test_func(x)) for x in np.linspace(0,0.5,1000)]

In [17]:
data_input = list(map(lambda x: [x[0]], data))
data_input_poly = list(map(lambda x: [x[0], x[0]**2, x[0]**3, x[0]**4], data))
data_target = list(map(lambda x: [x[1]], data))

In [18]:
def normal_nn(optim):
    data_input = tf.placeholder(tf.float32, shape=[None, 1])
    data_target = tf.placeholder(tf.float32, shape=[None, 1])
    
    poly = tf.concat(1, [data_input, data_input**2, data_input**3, data_input**4])
    net = tflearn.fully_connected(poly, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
    
    data_output = tflearn.fully_connected(net, 1, activation="linear")
    
    loss = tf.reduce_mean(tf.square(data_output - data_target))
    op = optim.minimize(loss)
    
    return op, data_input, data_target, loss, data_output

In [19]:
batch_size = 128

In [20]:
epoch_loss = []
nn_epoch_loss = []

In [21]:
epochs = 2000 

In [22]:
test_set = [[x] for x in np.linspace(-1,1,1000)]
test_set_poly = [[x,x**2,x**3,x**4] for x in np.linspace(-1,1,1000)]
test_set_flat = [a[0] for a in test_set]
data_input_flat = [a[0] for a in data_input]
data_target_flat = [a[0] for a in data_target]

In [23]:
op, data_x, data_y, loss, data_output = normal_nn(optim)
sess.run(tf.global_variables_initializer())
for e in tqdm(range(1000 * 2)):
    indices = np.random.randint(low=0, high=1000, size=batch_size)
    input_batch = [data_input[i] for i in indices]
    target_batch = [data_target[i] for i in indices]

    _, l = sess.run([op, loss], feed_dict={data_x: input_batch, data_y:target_batch, lr:0.001})
    nn_epoch_loss.append(l)

100%|██████████| 2000/2000 [00:00<00:00, 2200.26it/s]


In [24]:
nn_preds = sess.run(data_output, feed_dict={data_x: test_set})

In [25]:
# op, data_x, data_y, m_scaling, b_size, loss, kl_loss, data_loss, grads = bnn.update_v2(8, optim)
# sess.run(tf.global_variables_initializer())

In [26]:
bnn_data_input = tf.placeholder(tf.float32, shape=[None, 4])
bnn_data_target = tf.placeholder(tf.float32, shape=[None, 1])
kls = tf.placeholder(tf.float32, shape=[])

bnet_loss, klloss, dataloss = bnet.loss(bnn_data_input, bnn_data_target,kl_scaling=kls, N=8)
bnet_output = bnet.sample(bnn_data_input)

copy_old = bnet.copy_variational_parameters()
kl_div = bnet.kl_new_and_old()

grads_to_apply = optim.compute_gradients(bnet_loss)
# print(grads_to_apply)

clipped_policy_grads = []
for (grad, var) in grads_to_apply:
    if grad is not None:
        clipped_policy_grads.append((tf.clip_by_norm(grad, 1), var))

# clipped_policy_grads = grads_to_apply
grad_op = optim.apply_gradients(clipped_policy_grads)

sess.run(tf.global_variables_initializer())

In [27]:
bnn_lr = 0.001

In [28]:
epoch_loss=[]
kl_loss=[]
data_loss=[]
kl_divs=[]

In [29]:
for e in tqdm(range(epochs*10)):
#     input_batch = data_input_poly
#     target_batch = data_target
    if e > epochs*10*0.5:
        hh = 1000
    else:
        hh = 500
    indices = np.random.randint(low=0, high=hh, size=batch_size)
    input_batch = [data_input_poly[i] for i in indices]
    target_batch = [data_target[i] for i in indices]
    
#     kl = 0.0
    kl = batch_size/1000
#     kl = 1.0
#     kl = (2**(epochs - e - 1))/( (2**epochs) - 1) 

#     kl = 0.0

    _, l, kloss, dloss = sess.run([grad_op, bnet_loss, klloss, dataloss], feed_dict={kls:kl, bnn_data_input: input_batch, bnn_data_target:target_batch, lr:bnn_lr})
    epoch_loss.append(l)
    
    if e % 50 == 0:
        div = sess.run(kl_div)
        sess.run(copy_old)
        kl_divs.append(div)
    
    if e % 5 == 0:
        epoch_loss.append(l) 
        kl_loss.append(kloss)
        data_loss.append(dloss)

100%|██████████| 20000/20000 [00:47<00:00, 422.30it/s]


In [30]:
sess.run(bnet.layers[0].qw_sigma)

array([[ 0.70907795,  0.66980571,  0.71410382,  0.69864392,  0.64800733,
         0.6205231 ,  0.71327931,  0.69421959,  0.67634952,  0.6973812 ,
         0.69584364,  0.66283756,  0.67366773,  0.5142737 ,  0.69755536,
         0.68202186,  0.01643665,  0.00566544,  0.68707156,  0.65100265,
         0.59765851,  0.63257176,  0.68134099,  0.70572257,  0.01142118,
         0.6968171 ,  0.63517463,  0.6911661 ,  0.70516819,  0.66411293,
         0.69115728,  0.0074245 ],
       [ 0.92115152,  0.90549326,  0.91665763,  0.91734797,  0.90325534,
         0.88495463,  0.93199575,  0.91521955,  0.9075352 ,  0.91183335,
         0.90957129,  0.90484941,  0.90254474,  0.81221747,  0.91410959,
         0.91195339,  0.08896028,  0.01934168,  0.91152823,  0.89715278,
         0.88059831,  0.88483959,  0.91266346,  0.9246242 ,  0.03154158,
         0.9229055 ,  0.88911057,  0.91288757,  0.92749852,  0.90086985,
         0.91374213,  0.03098688],
       [ 0.98064911,  0.97737825,  0.97818881,  0.9855

In [31]:
# for e in tqdm(range(epochs)):
# #     indices = np.random.randint(low=0, high=1000, size=batch_size)
# #     input_batch = [data_input_poly[i] for i in indices]
# #     target_batch = [data_target[i] for i in indices]
#     input_batch = data_input_poly
#     target_batch = data_target
    
# #     ms = (2**(epochs - e - 1))/( (2**epochs) - 1) 
# #     ms = 1000/(batch_size)
# #     ms = (1000*epochs)/(batch_size*(e+1))
# #     ms = (1000)/(batch_size + e)
#     ms = 0
# #     ms = 1.0
    
# #     grad_nums = [a for (a,b) in grads]
#     qw_mean, qw_p, qb_mean, qb_p = sess.run([bnn.qw_means[0], bnn.qw_ps[0], bnn.qb_means[0], bnn.qb_ps[0]])
#     _, l, k_loss, d_loss, gg = sess.run([op, loss, kl_loss, data_loss, grads], feed_dict={data_x: input_batch, data_y:target_batch, m_scaling:ms, b_size:batch_size, lr:bnn_lr})
# #     print("----------Epoch {}-----------".format(e))
# #     print("{}\n{}\n".format(qb_mean, qb_p))
# #     print(gg[2:])
# #     print()
# #     print("KL:{}, Data:{}\n".format(k_loss, d_loss))
#     if e % 5 == 0:
#         epoch_loss.append(l)   

In [32]:
# sess.run(bnn.qw_ps[0])

In [33]:
# sess.run(bnn.qw_ps[0])
# grads = tf.gradients(loss, w_vars)
# sess.run(w_vars, feed_dict={data_x: data_input_poly, data_y:target_batch, m_scaling:[0.1]})

In [34]:
# pred_input = tf.placeholder(tf.float32, shape=[None, 4])
# pred_output = bnn.sampled_output(pred_input, 1000)
# pred_input, pred_output = bnn.sample(use_mean=False)
predictions = sess.run(bnet_output, feed_dict={bnn_data_input: test_set_poly})
var_trials = 500
for _ in tqdm(range(var_trials-1)):
    predictions = np.concatenate([predictions, sess.run(bnet_output, feed_dict={bnn_data_input: test_set_poly})], axis=1)

100%|██████████| 499/499 [00:00<00:00, 592.24it/s]


In [35]:
pred_means = np.apply_along_axis(lambda x: np.mean(x), 1, predictions)
pred_means[3]

0.82763088

In [36]:
pred_vars = np.apply_along_axis(lambda x: np.std(x, ddof=1), 1, predictions)

In [37]:
predictions.shape

(1000, 500)

In [38]:
p = figure(width=800, height=800, y_range=(-0.5,1.5), x_range=(-1,1))

p.line(data_input_flat, data_target_flat, line_width=0.5, color=(0,0,255,0.5))

err_xs=[]
err_ys=[]
for x, y, yerr in zip(test_set_flat, pred_means, pred_vars):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

p.multi_line(err_xs, err_ys, color=(255,0,0,0.15))

# p.line([a/100 for a in range(100)], [scale*a/100 for a in range(100)], line_width=3, color="blue")
p.line(test_set_flat, pred_means, line_width=2, color="red")

for i in range(0, 200, 25):
    pp = list(map(lambda x: x[i], predictions))
    p.line(test_set_flat, pp, line_width=0.5, color=(255,0,255,0.2))

# p.line(data_input_flat, data_target_flat, line_width=0.5, color=(0,0,255,0.5))

p.line(test_set_flat, nn_preds.flatten(), line_width=2, color="orange")
# Loss
# p.line(range(epochs), epoch_loss, line_width=2, color="green")

show(p)

In [39]:
loss_p = figure(width=500, height=500)

# Loss
# loss_p.line(range(len(epoch_loss)), epoch_loss, line_width=0.5, color="green")

loss_p.line(range(len(kl_loss)), kl_loss, line_width=0.5, color="orange")

loss_p.line(range(len(data_loss)), data_loss, line_width=0.5, color="purple")

# loss_p.line(range(len(nn_epoch_loss)), nn_epoch_loss, line_width=1, color="green")

show(loss_p)

In [40]:
loss_kl = figure(width=500, height=500)

loss_kl.line(range(len(kl_divs)), kl_divs, line_width=0.5, color="red")

show(loss_kl)

In [41]:
kl_divs

[0.064219594,
 23.708939,
 17.976284,
 12.723122,
 12.358714,
 10.161222,
 8.8096886,
 9.5339441,
 11.069159,
 8.7510729,
 7.202095,
 8.6359053,
 9.5298567,
 8.0152464,
 6.2720518,
 6.367835,
 5.2347794,
 4.7767191,
 4.8862753,
 4.8722486,
 4.7345886,
 4.2741704,
 4.3016596,
 4.0996976,
 4.0755196,
 3.4857733,
 3.5568554,
 3.8476386,
 3.0619297,
 3.45717,
 3.1112523,
 3.5135241,
 3.4692893,
 3.1182375,
 3.3147531,
 3.7736225,
 4.8347545,
 5.3639035,
 4.5648718,
 3.6523728,
 3.479346,
 3.6974738,
 4.276536,
 2.6786284,
 3.995784,
 5.3000731,
 3.6677251,
 3.2721708,
 4.5489616,
 4.8302245,
 3.9303336,
 4.7468033,
 4.1888075,
 2.5708833,
 3.1503601,
 2.8382697,
 2.6691079,
 4.6462708,
 4.9043255,
 4.1590528,
 1.0140083,
 2.2682424,
 1.8514143,
 2.5255606,
 2.7674365,
 3.2840707,
 2.2403843,
 1.7588253,
 1.2703245,
 1.2344315,
 1.6304377,
 1.4411726,
 1.3712931,
 1.2183609,
 1.4052024,
 1.1691623,
 2.9729381,
 0.77381438,
 1.4190245,
 0.79918993,
 2.0956059,
 1.0482107,
 2.256918,
 3.76599